In [ ]:
# Cell: Install Required Libraries
!pip install -q langchain_dartmouth PyPDF2 tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/232.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
# Cell: Load API keys and model from config file
import os
try:
    import config
    os.environ["DARTMOUTH_API_KEY"] = config.DARTMOUTH_API_KEY
    os.environ["DARTMOUTH_CHAT_API_KEY"] = config.DARTMOUTH_CHAT_API_KEY
    SELECTED_MODEL = getattr(config, "SELECTED_MODEL", "openai.gpt-5-mini-2025-08-07")
    print("API keys loaded from config.")
except ImportError as e:
    raise RuntimeError("Please create a config.py file with your API keys as shown in config_template.py")


Enter DARTMOUTH_API_KEY: ··········
Enter DARTMOUTH_CHAT_API_KEY: ··········
Keys set: True True
Selected model: openai.gpt-5-mini-2025-08-07


In [ ]:
# Read local case study documents (Word files) instead of using local Drive/Docs
import docx
import os

BASE_FOLDER = '../data'
CLEAN_CASE_STUDY_PATH = os.path.join(BASE_FOLDER, 'clean_case_studies.docx')
ADVERSARIAL_CASE_STUDY_PATH = os.path.join(BASE_FOLDER, 'adversarial_case_studies.docx')


def read_docx(path):
    doc = docx.Document(path)
    return '
'.join([para.text for para in doc.paragraphs])

clean_case_study_text = read_docx(CLEAN_CASE_STUDY_PATH)
adversarial_case_study_text = read_docx(ADVERSARIAL_CASE_STUDY_PATH)

print('Loaded clean case study text (length):', len(clean_case_study_text))
print('Loaded adversarial case study text (length):', len(adversarial_case_study_text))

Mounted at /content/drive
BASE_FOLDER   → /content/drive/MyDrive/Ophthalmology Live Folder
OUTPUT_FOLDER → /content/drive/MyDrive/Ophthalmology Live Folder/Case Study Output
CLEAN DOC ID  → 1xoVkMUsRF8ozwoBnq9QSJduZcDuAtgE58rn5dqhoPKw
ADV DOC ID    → 1ayKJwGEAG-yPTpz2yyLo0pPSGqnTW4fMcMNeqWdwPVs


In [ ]:
# Helpers for local Docs removed since we use local files.

In [ ]:
# Cell: Parse cases from local Word document instead of local Doc
import re
from collections import OrderedDict

def extract_cases_from_docx_text(text: str):
    """Parse the combined text of a local Word doc into case dictionary."""
    cases = OrderedDict()
    # Pattern matches "CASE" followed by number and dash or em-dash
    pattern = re.compile(r"CASE\s*(\d+)\s*[—-]", re.IGNORECASE)
    parts = pattern.split(text)
    # parts[0] is header/preamble; then pairs of number and content
    for i in range(1, len(parts), 2):
        case_num = parts[i].strip()
        case_text = parts[i+1].strip() if (i+1) < len(parts) else ""
        cases[case_num] = case_text
    return cases

# Example usage:
# raw_text = read_docx(CLEAN_CASE_STUDY_PATH)
# cases = extract_cases_from_docx_text(raw_text)
# print(cases.keys())


Found 98 clean cases and 98 adversarial cases.
Example CLEAN case: CASE 1: Acute Follicular Conjunctivitis
Example ADV case: CASE 1: Acute Follicular Conjunctivitis


In [ ]:
# Cell 6: Field and Section Definitions
SUBJECTIVE_FIELDS_1 = [
    "CHIEF COMPLAINT", "DURATION", "LATERALITY", "PATIENT QUOTES",
    "HISTORY OF PRESENT ILLNESS", "History Narrative", "Symptom Onset",
    "Symptom Duration", "Progression", "Characteristics"
]
SUBJECTIVE_FIELDS_2 = [
    "Aggravating Factors", "Alleviating Factors", "Associated Symptoms", "Previous Episodes",
    "Prior Treatments", "Impact on Daily Activities", "Systemic Symptoms",
    "Recent Travel", "Contacts with Similar Symptoms", "Risk Factors"
]
OBJECTIVE_FIELDS_1 = [
    "Vitals – Blood Pressure", "Vitals – Pulse", "Vitals – Temperature", "Vitals – Respirations",
    "Visual Acuity – Uncorrected Right", "Visual Acuity – Corrected Right",
    "Visual Acuity – Uncorrected Left", "Visual Acuity – Corrected Left",
    "Visual Acuity – Near Acuity", "Visual Acuity – Method",
    "Intraocular Pressure – Right", "Intraocular Pressure – Left", "Intraocular Pressure – Method"
]
OBJECTIVE_FIELDS_2 = [
    "External Exam – Lids/Lashes", "External Exam – Conjunctiva/Sclera", "External Exam – Lacrimal System", "External Exam – Orbit",
    "Anterior Segment – Cornea", "Anterior Segment – Anterior Chamber", "Anterior Segment – Iris",
    "Anterior Segment – Angle", "Anterior Segment – Lens"
]
OBJECTIVE_FIELDS_3 = [
    "Pupils – Size/Shape", "Pupils – Reaction to Light", "Pupils – Reaction to Near", "Pupils – RAPD", "Pupils – Other",
    "Extraocular Movements", "Slit Lamp Exam Description",
    "Fundus Exam – Optic Nerve", "Fundus Exam – Macula", "Fundus Exam – Vessels", "Fundus Exam – Periphery"
]
ASSESSMENT_FIELDS = [
    "Assessment Narrative", "Primary Diagnosis", "Secondary Diagnoses",
    "Differential Diagnosis", "Severity", "Laterality", "Complications"
]
PLAN_FIELDS = [
    "All Medications", "All Procedures", "Follow-Up Instructions",
    "Patient Education", "Referrals", "Work Restrictions", "Emergency Instructions"
]
REQUIRED_HEADERS = {
    "subjective1": SUBJECTIVE_FIELDS_1,
    "subjective2": SUBJECTIVE_FIELDS_2,
    "objective1": OBJECTIVE_FIELDS_1,
    "objective2": OBJECTIVE_FIELDS_2,
    "objective3": OBJECTIVE_FIELDS_3,
    "assessment": ASSESSMENT_FIELDS,
    "plan": PLAN_FIELDS,
}
SECTION_ORDER = [
    "subjective1", "subjective2",
    "objective1", "objective2", "objective3",
    "assessment", "plan"
]
SECTION_LABELS = {
    "subjective1": "SUBJECTIVE",
    "subjective2": "SUBJECTIVE (cont.)",
    "objective1": "OBJECTIVE EXAM",
    "objective2": "OBJECTIVE EXAM (cont.)",
    "objective3": "OBJECTIVE EXAM (cont. 2)",
    "assessment": "ASSESSMENT",
    "plan": "PLAN"
}

In [ ]:
# Cell 7: Run configuration with manual start indices and optional limits
from tqdm import tqdm

# Which sets to run
RUN_CLEAN = True
RUN_ADVERSARIAL = True

# Manual start points (case index numbers from the docs)
START_CLEAN_AT = 1        # e.g., 40 to start at CLEAN case 40
START_ADVERSARIAL_AT = 1  # e.g., 50 to start at ADV case 50

# Optional limits (None means no limit) — helpful for quick tests
MAX_CASES_CLEAN = 98       # e.g., 1–3 for testing; set None to run all
MAX_CASES_ADVERSARIAL = 98

# Checkpoint behavior
SKIP_COMPLETED = True      # skip cases already complete in checkpoint
RESET_CHECKPOINTS = True  # set True once to clear old checkpoints

# LLM retry config
MAX_RETRIES = 10
PRINT_FAILED_OUTPUTS = True

In [ ]:
# Cell 8: LLM client, UTC timestamp, quota detection, smoke test
import time, random, json
from datetime import datetime, timezone
try:
    from datetime import UTC  # Py 3.12+
except ImportError:
    UTC = timezone.utc

from langchain_dartmouth.llms import ChatDartmouthCloud

# gpt-5 requires temperature=1
GENERATION_KW = dict(temperature=1)

def now_iso():
    return datetime.now(UTC).replace(microsecond=0).isoformat().replace("+00:00", "Z")

def get_llm():
    # Always and only use your selected model, with required params
    return ChatDartmouthCloud(model_name=SELECTED_MODEL, temperature=1)

def call_llm_text(llm, messages, retries=MAX_RETRIES, sleep_base=1.2):
    last_exc = None
    for attempt in range(1, retries+1):
        try:
            resp = llm.invoke(messages, **GENERATION_KW)
            # Common response shapes
            if isinstance(resp, str):
                return resp.strip()
            if hasattr(resp, "content") and isinstance(resp.content, str):
                return resp.content.strip()
            if isinstance(resp, dict):
                if isinstance(resp.get("content"), str):
                    return resp["content"].strip()
                if isinstance(resp.get("text"), str):
                    return resp["text"].strip()
            if hasattr(resp, "generations"):
                return resp.generations[0][0].text.strip()
            text = str(resp).strip()
            if text:
                return text
            raise RuntimeError("Empty response")
        except Exception as e:
            last_exc = e
            time.sleep(sleep_base * (1.5 ** (attempt-1)) + random.random())
    raise last_exc

def is_quota_or_rate_error(exc: Exception) -> bool:
    s = str(exc).lower()
    return any(k in s for k in [
        "insufficient_quota", "quota", "rate limit", "429",
        "out of tokens", "insufficient tokens", "credit",
        "payment_required", "maximum context length", "context_length_exceeded",
        "token limit"
    ])

# Smoke test (fail fast if model is not reachable or params rejected)
print("Running smoke test against:", SELECTED_MODEL)
try:
    probe = get_llm()
    msg = [
        {"role": "system", "content": "You are a test probe."},
        {"role": "user", "content": "Reply with exactly: OK"}
    ]
    out = call_llm_text(probe, msg, retries=1)
    print("Smoke test response:", out)
except Exception as e:
    print("SMOKE TEST FAILED for model:", SELECTED_MODEL)
    raise

Running smoke test against: openai.gpt-5-mini-2025-08-07
Smoke test response: OK


In [ ]:
# Cell 9: Checkpoint utilities (per-field durable save)
import os

def sanitize_filename(s: str) -> str:
    return "".join(ch if ch.isalnum() or ch in ("-", "_", " ", ".") else "_" for ch in s).strip()

def checkpoint_path(out_prefix: str) -> str:
    return os.path.join(
        OUTPUT_FOLDER,
        f"checkpoint_{out_prefix}_{sanitize_filename(SELECTED_MODEL)}.json"
    )

def load_checkpoint(out_prefix: str) -> dict:
    path = checkpoint_path(out_prefix)
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        data.setdefault("model", SELECTED_MODEL)
        data.setdefault("out_prefix", out_prefix)
        data.setdefault("cases", {})
        return data
    return {"model": SELECTED_MODEL, "out_prefix": out_prefix, "cases": {}}

def save_checkpoint(out_prefix: str, ckpt: dict):
    path = checkpoint_path(out_prefix)
    tmp = path + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(ckpt, f, ensure_ascii=False, indent=2)
    os.replace(tmp, path)

def case_key(case_index: int, case_title: str) -> str:
    return f"Case {case_index:02d} {case_title}"

In [ ]:
# Cell 10: Field extraction with concise instructions and per-field checkpoint saves

def assemble_note_from_fields_minimal(fields_dict: dict) -> str:
    # Minimal assembly for intermediate saves; final doc rendering is applied later
    out_sections = []
    for section in SECTION_ORDER:
        fields = REQUIRED_HEADERS[section]
        lines = [f"{field}: {fields_dict.get(field, 'N/A')}" for field in fields]
        out_sections.append(SECTION_LABELS.get(section, section).upper() + "\n" + "\n".join(lines))
    return "\n\n".join(out_sections)

def extract_fields_for_case(llm, case_text: str, out_prefix: str, k: str, starting_fields: dict = None, verbose=False) -> dict:
    """
    Extracts fields one-by-one, saving checkpoint after each field.
    On quota/rate error, saves partial and raises to stop gracefully.
    Prompts tuned to be medically accurate AND concise (avoid laundry lists).
    """
    ckpt = load_checkpoint(out_prefix)
    fields_done = dict(starting_fields or {})

    # Ordered list of fields
    all_fields_in_order = []
    for section in SECTION_ORDER:
        all_fields_in_order.extend(REQUIRED_HEADERS[section])

    for field in all_fields_in_order:
        if field in fields_done and (fields_done[field] or fields_done[field] == "N/A"):
            continue

        if field in OBJECTIVE_FIELDS_1:
            system_line = (
                "You are an ophthalmology scribe. "
                "For the field below, output a medically plausible, typical, or case-appropriate value even if not explicitly stated. "
                "Only respond with 'N/A' if it is truly impossible to infer a reasonable value."
            )
        elif field in PLAN_FIELDS:
            system_line = (
                "You are a professional ophthalmology medical scribe. "
                "Return ONLY the value for the named field. Be concise. "
                "Prioritize items explicitly stated in the case; if you infer typical care, include at most one core, standard-of-care item. "
                "Avoid exhaustive or speculative lists. If truly unknown or not applicable, reply 'N/A'."
            )
        elif field == "Differential Diagnosis":
            system_line = (
                "You are a professional ophthalmology medical scribe. "
                "Return ONLY the value for 'Differential Diagnosis'. Be concise and list the 2–4 most likely differentials; "
                "separate them with semicolons. If not applicable, reply 'N/A'."
            )
        else:
            system_line = (
                "You are a professional ophthalmology medical scribe. "
                "Given the case below, infer ONLY the value for the named field. "
                "If not explicitly stated but typical or reasonably deducible, provide a concise inference. "
                "Use 'N/A' strictly if not relevant or impossible to infer."
            )

        user_line = (
            f"Case:\n{case_text}\n\n"
            f"Field to extract: {field}\n"
            f"Return only the value for this field, no label or extra text."
        )
        messages = [
            {"role": "system", "content": system_line},
            {"role": "user", "content": user_line}
        ]

        try:
            ans = call_llm_text(llm, messages)
            clean_ans = (ans or "").strip().replace("\n", " ")
            if not clean_ans:
                clean_ans = "N/A"
            fields_done[field] = clean_ans
            if verbose:
                print(f"{field}: {clean_ans}")
        except Exception as e:
            if is_quota_or_rate_error(e):
                # Save partial progress and stop
                case_entry = ckpt["cases"].setdefault(k, {"status": "partial", "fields": {}, "title": k, "last_updated": now_iso()})
                case_entry["fields"].update(fields_done)
                case_entry["status"] = "partial"
                case_entry["last_updated"] = now_iso()
                case_entry["note"] = assemble_note_from_fields_minimal(case_entry["fields"])
                save_checkpoint(out_prefix, ckpt)
                raise
            else:
                if PRINT_FAILED_OUTPUTS:
                    print(f"Non-quota exception for {k} – {field}: {e}")
                fields_done[field] = "N/A"

        # Save after each field
        case_entry = ckpt["cases"].setdefault(k, {"status": "partial", "fields": {}, "title": k, "last_updated": now_iso()})
        case_entry["fields"].update(fields_done)
        case_entry["status"] = "partial"
        case_entry["last_updated"] = now_iso()
        case_entry["note"] = assemble_note_from_fields_minimal(case_entry["fields"])
        save_checkpoint(out_prefix, ckpt)

    return fields_done

In [ ]:
# Cell 11: Human-readable rendering (no bullets; always show headers; omit N/A lines)
# Also write both local Doc and combined .txt file

PRETTY_SECTION_LABELS = {
    "subjective1": "Subjective",
    "subjective2": "Subjective (cont.)",
    "objective1": "Objective Exam",
    "objective2": "Objective Exam (cont.)",
    "objective3": "Objective Exam (cont. 2)",
    "assessment": "Assessment",
    "plan": "Plan",
}

def _normalize_value(value: str) -> str:
    """
    Normalize a field value for one-line plain text:
    - Treat empty or 'N/A' as empty (to be omitted)
    - Collapse whitespace/newlines to single spaces
    """
    if not value:
        return ""
    if value.strip().upper() == "N/A":
        return ""
    return " ".join(value.split())

def pretty_note_for_plain_text(fields_dict: dict) -> str:
    """
    Render a single case note as realistic plain text (no bullets):
    - Always show section headers
    - Only include non-N/A field lines
    - Each field is "Label: value" on its own line
    - Blank line after each section
    """
    parts = []
    for section in SECTION_ORDER:
        header = PRETTY_SECTION_LABELS.get(section, SECTION_LABELS.get(section, section))
        parts.append(header)  # always show header

        fields = REQUIRED_HEADERS[section]
        for f in fields:
            val = _normalize_value(fields_dict.get(f, ""))
            if not val:
                continue  # omit N/A/empty
            parts.append(f"{f}: {val}")
        parts.append("")  # blank line after section
    return "\n".join(parts).strip()

def rebuild_case_text_from_ckpt_entry(entry: dict) -> str:
    fields = entry.get("fields")
    if isinstance(fields, dict) and fields:
        return pretty_note_for_plain_text(fields)
    return _normalize_value(entry.get("note", "") or "")

def combined_text_from_completed(out_prefix: str) -> str | None:
    ckpt = load_checkpoint(out_prefix)
    done = []
    for k, v in ckpt["cases"].items():
        if v.get("status") == "complete":
            done.append((v.get("case_index", 9999), k, v))
    if not done:
        print(f"[{out_prefix}] No completed cases to output.")
        return None

    done.sort(key=lambda x: (x[0], x[1]))

    doc_parts = []
    for case_index, key, entry in done:
        # Case title like "CASE NN — Title"
        try:
            tokens = key.split(" ", 3)
            if len(tokens) >= 3 and tokens[0].lower() == "case":
                num = tokens[1]
                title = key.split(" ", 2)[-1]
                if title[:2] == num:
                    title = title[2:].strip()
                case_title_line = f"CASE {num} — {title}"
            else:
                case_title_line = key
        except Exception:
            case_title_line = key

        underline = "=" * len(case_title_line)
        doc_parts.append(case_title_line)
        doc_parts.append(underline)
        doc_parts.append(rebuild_case_text_from_ckpt_entry(entry))
        doc_parts.append("")  # blank between cases

    return "\n".join(doc_parts).strip()

def write_combined_to_doc(out_prefix: str, doc_id: str):
    combined_text = combined_text_from_completed(out_prefix)
    if combined_text:
        replace_doc_with_text(doc_id, combined_text)
        print(f"[{out_prefix}] Updated local Doc with clean plain-text output.")

def write_combined_to_txt(out_prefix: str, filename: str | None = None) -> str | None:
    combined_text = combined_text_from_completed(out_prefix)
    if not combined_text:
        return None
    if filename is None:
        filename = f"{out_prefix}_soap_notes_{sanitize_filename(SELECTED_MODEL)}.txt"
    path = os.path.join(OUTPUT_FOLDER, filename)
    with open(path, "w", encoding="utf-8") as f:
        f.write(combined_text)
    print(f"[{out_prefix}] Wrote plain-text file:", path)
    return path

In [ ]:
# Cell 12: Orchestrate runs with manual start, checkpointing, and outputs (Doc + TXT)

def run_cases_with_manual_start(case_dict, out_prefix: str, target_doc_id: str | None = None, start_index: int = 1, max_cases=None):
    # Reset checkpoint if requested
    if RESET_CHECKPOINTS:
        p = checkpoint_path(out_prefix)
        if os.path.exists(p):
            os.remove(p)
            print(f"Deleted old checkpoint: {p}")

    # Items filtered by start_index and optional max_cases
    items = [(idx, data) for idx, data in case_dict.items() if idx >= start_index]
    items.sort(key=lambda x: x[0])
    if max_cases is not None:
        items = items[:max_cases]

    llm = get_llm()
    ckpt = load_checkpoint(out_prefix)
    completed_keys = {k for k, v in ckpt["cases"].items() if v.get("status") == "complete"} if SKIP_COMPLETED else set()

    print(f"{out_prefix}: starting at case index {start_index}. Total to process this run: {len(items)}")
    try:
        for idx, case in tqdm(items, desc=f"{SELECTED_MODEL} {out_prefix}", position=0):
            k = case_key(idx, case['title'])
            if SKIP_COMPLETED and k in completed_keys:
                continue

            # Resume partial fields if present
            existing = ckpt["cases"].get(k, {})
            starting_fields = existing.get("fields", {}) if existing.get("status") in ("partial", "complete") else {}

            fields_done = extract_fields_for_case(llm, case['text'], out_prefix, k, starting_fields=starting_fields, verbose=False)

            # Mark complete in checkpoint (final text re-rendered later for Doc/TXT)
            ckpt = load_checkpoint(out_prefix)
            ckpt["cases"][k] = {
                "status": "complete",
                "fields": fields_done,
                "title": k,
                "last_updated": now_iso(),
                "note": assemble_note_from_fields_minimal(fields_done),
                "case_index": idx
            }
            save_checkpoint(out_prefix, ckpt)

        print(f"\n[{out_prefix}] Finished processing selected cases.")
    except Exception as e:
        if is_quota_or_rate_error(e):
            print(f"\n[{out_prefix}] Stopping due to quota/rate limit: {e}")
        else:
            print(f"\n[{out_prefix}] Unexpected error: {e}")

    # Push current completed results to the target local Doc and write TXT mirror
            # Only update the local Doc if a doc_id is provided
        if target_doc_id:
            write_combined_to_doc(out_prefix, target_doc_id)
    write_combined_to_txt(out_prefix)

In [ ]:
# Cell 13: Execute runs using your manual start points and limits

if RUN_CLEAN and clean_cases:
    run_cases_with_manual_start(
        clean_cases,
        out_prefix="CLEAN",
        start_index=START_CLEAN_AT,
        max_cases=MAX_CASES_CLEAN
    )

if RUN_ADVERSARIAL and adversarial_cases:
    run_cases_with_manual_start(
        adversarial_cases,
        out_prefix="ADVERSARIAL",
        start_index=START_ADVERSARIAL_AT,
        max_cases=MAX_CASES_ADVERSARIAL
    )

Deleted old checkpoint: /content/drive/MyDrive/Ophthalmology Live Folder/Case Study Output/checkpoint_CLEAN_openai.gpt-5-mini-2025-08-07.json
CLEAN: starting at case index 1. Total to process this run: 98


openai.gpt-5-mini-2025-08-07 CLEAN: 100%|██████████| 98/98 [4:51:02<00:00, 178.19s/it]



[CLEAN] Finished processing selected cases.
Replaced content of Google Doc 1xoVkMUsRF8ozwoBnq9QSJduZcDuAtgE58rn5dqhoPKw (771571 chars).
[CLEAN] Updated Google Doc with clean plain-text output.
[CLEAN] Wrote plain-text file: /content/drive/MyDrive/Ophthalmology Live Folder/Case Study Output/CLEAN_soap_notes_openai.gpt-5-mini-2025-08-07.txt
Deleted old checkpoint: /content/drive/MyDrive/Ophthalmology Live Folder/Case Study Output/checkpoint_ADVERSARIAL_openai.gpt-5-mini-2025-08-07.json
ADVERSARIAL: starting at case index 1. Total to process this run: 98


openai.gpt-5-mini-2025-08-07 ADVERSARIAL: 100%|██████████| 98/98 [4:39:04<00:00, 170.86s/it]



[ADVERSARIAL] Finished processing selected cases.
Replaced content of Google Doc 1ayKJwGEAG-yPTpz2yyLo0pPSGqnTW4fMcMNeqWdwPVs (727846 chars).
[ADVERSARIAL] Updated Google Doc with clean plain-text output.
[ADVERSARIAL] Wrote plain-text file: /content/drive/MyDrive/Ophthalmology Live Folder/Case Study Output/ADVERSARIAL_soap_notes_openai.gpt-5-mini-2025-08-07.txt
